In [ ]:
import os
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler, QuantileTransformer

import numpy as np
from scipy.optimize import minimize

from bokeh.plotting import figure, show, output_notebook, gridplot
from scipy.stats import linregress
from bokeh.models import ColumnDataSource, Whisker, FactorRange
from bokeh.layouts import grid, row, column
from bokeh.transform import factor_cmap, jitter
import seaborn as sns
from bokeh.application import Application
from bokeh.application.handlers import FunctionHandler


import holoviews as hv
from holoviews import opts
hv.extension('bokeh')
output_notebook()

# Overview 



We compare distributions of unit area runoff between pairs of measured locations to see if the "closeness" of the distributions can be predicted from a large set of attributes that describe the terrain, soil, land cover, and climate of each basin.  The analysis involves two main components/models:  
* the quantization used to transform time series streamflow observations into discrete probability distributions, and
* the Neural Network (NN) model used to predict the measure of difference between pairs of distributions from basin attributes.

##  Limitations
* Index representation of attributes,
* Skewed distribution of basin attributes (drainage area is exponentially distributed),
* Resolution of geospatial data (climate, $1 km$) compared to basin scale (min $1 km^2$),
* Measurement uncertainty in runoff data being greater than reported precision.
* Sample of observed locations is unevenly distributed in space, with a strong bias to the south of the province, meaning certain regions with unique hydrometeorological conditions are under-represented.
* Minimum concurrent period of record for basin pair selection criteria leaves out valuable information by excluding comparisons and unique stations.
* Maxiximum basin centroid distance adds selection bias, but excludes excess comparisons that might otherwise dilute the sample from more reasonable comparisons. I.e. allowing comparison of all pairs vs. comparison of mostly reasonable pairings that would easily be excluded, or artificially diluting the maximum rang.  On the other hand, the data should speak for itself?
* attributes are probably not completely orthogonal (correlation between attributes)

1) test how often the lowest DKL corresponds to the closest pair. (each basin gets compared with many other basins).
2) add attributes one at a time and see how often the lowest DKL corresponds to the closest pair in (increasing dimension space).


## Looking to Graph Theory

>Let a graph G of size N represent a network of two classes of nodes, 1 (blue) and 2 (green), where green is sparse in blue.  Let the edges (node connectivity) of a network graph G represent the shortest distance in B-dimensional space between each blue-green pair, where B is a set of unique attributes used to describe each node.   Let M represent a B by N matrix (adjacency) describing the connectivity of graph G.  What can graph theory offer to help interpret and understand how the graph connectivity changes as we vary the subset of attributes B used to compute the shortest distance?

1. **Changes in Graph Topology**: By varying B, you may observe changes in the graph's topology, such as the formation or dissolution of edges, changes in the number of connected components, or the emergence of new clusters. Graph theory provides tools to quantify these changes, such as through the calculation of the number of connected components, the clustering coefficient, or measures of graph density.

2. **Shortest Path Analysis**: The subset B* of attributes B directly affects the computation of the shortest paths between nodes (blue-green pairs in your case). Algorithms like Dijkstra's or the Floyd-Warshall algorithm can be used to find these shortest paths. Changes in B could lead to variations in the shortest path lengths and the actual paths taken, which can be analyzed to understand how different attributes influence connectivity.

3. **Centrality Measures**: Graph theory offers various centrality measures (such as degree centrality, betweenness centrality, closeness centrality, and eigenvector centrality) to identify the most important nodes within the graph. Altering B could shift the centrality scores of nodes, indicating how changes in attribute consideration impact the relative importance or influence of nodes within the network.

4. **Community Detection**: By changing B, the community structure within the graph may also change. Graph theory provides methods for community detection and modularity optimization, such as the Louvain method or Girvan-Newman algorithm, which can help identify how groups of nodes (communities) are formed or dissolved based on the attributes considered.

5. **Robustness and Resilience**: The resilience of the network to failures or attacks can be assessed through graph theoretical measures. By varying B, you can study how the network's vulnerability to node or edge removal changes, providing insights into which attributes contribute to a more robust or fragile network structure.

6. **Spectral Analysis**: The eigenvalues and eigenvectors of the adjacency matrix or the Laplacian matrix of the graph can provide insights into the graph's properties, such as connectivity, the existence of bipartite structures, or the presence of community structures. The spectral properties can change with variations in B, offering a mathematical lens through which to view the impact of attributes on overall network structure.

7. **Visualization**: Graph theory supports the visualization of complex networks, allowing for the visual assessment of how changes in B affect the graph. Visual analysis can reveal patterns, clusters, outliers, or structural changes that are not immediately obvious through numerical analysis alone.


## Visualization Concept

* map of Vancouver Island.
* blue lines indicate rivers,
* triangles represent streamflow monitoring network,
* blue dots indicate decision space (ungauged locations)
  
* use a small subset of observation network (triangles)
* use unique colors for each triangle (station) to enhance visual contrast
* the above is the "basis arrangement"

* for each blue dot, find the "closest" station in 1d space  
    * distance is spatial (xy) distance between basin polygon centroids
* transform each blue dot into its corresponding proxy station shape/color
* return to the "basis arrangement"
* for each blue dot, repeat finding the "nearest" station in 2 to N dimensional space
 

```
Define ungauged basin attributes as A (N dimensional vector for each basin)
Define decision space as M
Define the set of streamflow monitoring locations as S.

For D in range(1...N):
    Define A_sub = A[1:D]
    For each blue dot c in (M not S):
        For each triangle t in S:
            connect a line from blue dot c to triangle t (compute pairwise distance to each station)
            emphasize the triangle representing the shortest distance

    Convert all blue dots into their proxy shapes/colours

    If D is not 1:
        Note any changes in chosen proxies
        store a visual snapshot of the proxy (store proxy rankings)

```

The above describes a network graph.  One issue is the ordering of A.  Is there an existing approach to evaluating how the graph connectivity changes as a function of the subset of A used to compute distance?



## Sensitivity Analysis

**Define a model arrangement to serve as a basis of comparison for sensitivity analysis.**

Test the sensitivity to quantization by: 
1) varying dictionary size (4, 5, 6, 7, 8 bits),
2) varying quantization approach (uniform & equiprobable),
3) keep track of $D_{KL}$ values by pairs across 1 and 2 and look for trends.

Test the NN model sensitivity to sample size:
1)  There are 1507 observed locations, and 2.4M pairs (bi-directional comparisons since $D_{KL}$ is not symmetrical).
2)  Of these pairs, ~150K have at least 10 years concurrent record and are not further than 500km apart (measured by basin polygon centroid distance).
3)  Train NN with 1500, 1000, and 500 stations, hold # pairs constant use MC simulation to express the range of outcomes.

Test the NN model sensitivity to sample bias in the geographic distribution of stations:
1) Training the NN with geographic bias
   * (leave out N from training set above 70 degrees North and hold
  
Test the sensitivity of NN structure:
1) Define a basis model (look to other studies)
2) Decrease 


## The Kullback-Leibler Divergence

The Kullback-Leibler divergence $D_{KL}$ is a measure of difference between two distributions.  The value of the measure represents the number of additional bits of memory needed to fully describe the observed data P (posterior/observed) given a model (prior/simulation) Q:

$$D_{KL}(P ‖ Q) = \sum_{i=0}^N P \cdot log \left(\frac{P}{Q} \right)$$


One issue with the measure occurs where distributions don't overlap, which is often in the case of streamflow.  In many cases, sum(P) < sum(Q) after dropping i where p | q = 0, and we are often left with Pi<Qi yielding DKL < 0.  Or we may be misled into thinking DKL is small if we don't actually check the distributions.

To get around this, my first instinct was to simply add 1 to the vector of bin counts (C* = C + [1,..., 1]), however the resulting C adds more information to the outliers.  It seems more reasonable to do C* = N*C + [1, ...,1]), where N is some multiple, I started with N=2.  This will change the entropy of both distributions, so I suppose we should track this change such that it can be compared at subsequent steps?  i.e. to ensure any effects are substantially greater than the information we've added.  We could also increase N, but I guess this represents a tradeoff in penalizing a certain characteristic of divergence vs. adding noise to the distributions.

## The Jensen-Shannon Divergence 

$$D_{JS}(P ‖ Q) = \frac{1}{2} D_{KL}(P ‖ M) + \frac{1}{2} D_{KL}(Q ‖ M)$$

$$D_{JS-\text{LOW FLOW}}(P_1 ‖ Q_1) = \frac{1}{2} D_{KL}(P_1 ‖ M_1) + \frac{1}{2} D_{KL}(Q_1 ‖ M_1)$$

$$D_{JS-\text{HIGH FLOW}}(P_N ‖ Q_N) = \frac{1}{2} D_{KL}(P_N ‖ M_N) + \frac{1}{2} D_{KL}(Q_N ‖ M_N) \text{ where } N =2^{\text{BITRATE}}$$

where D is the Kullback-Leibler divergence, and M is the average of P and Q, defineas:d 

asQ)(M = \frac{1}{2}(P + Q)$$

The Jensen-Shannon Divergence can be expressed as:

$$JSD(P ‖ Q) = \frac{1}{2} \sum_{i=0}^N P \cdot log \left( \frac{P}{M} \right) + \frac{1}{2} \sum_{i=0}^N Q \cdot log \left( \frac{Q}{M} \right)$$

In [ ]:
def reformulate_jdf_model_result(model, df, b):
    last_col = 2**b - 1
    jdf = df[f'jsd_{model}'].str.split(', ', expand=True)
    # print(jdf[[0, last_col]].head())
    jdf[0] = jdf[0].apply(lambda r: r.split('[')[1])
    jdf[last_col] = jdf[last_col].apply(lambda r: r.split(']')[0])
    
    for c in jdf.columns:
        jdf[c] = jdf[c].astype(float)
    jdf.columns = [f'{c:03d}' for c in jdf.columns]
    
    low_flow_cols = list(jdf.columns)[:2]
    high_flow_cols = list(jdf.columns)[-2:]
    mid_range_cols = [e for e in list(jdf.columns) if e not in low_flow_cols + high_flow_cols]
    for quantile_focus in [f'low_flow_jsd_{model}', f'mid_range_jsd_{model}', f'high_flow_jsd_{model}']:
        if quantile_focus.startswith('low'):
            df[quantile_focus] = jdf.loc[:, low_flow_cols].sum(1)
        elif quantile_focus.startswith('mid'):
            df[quantile_focus] = jdf.loc[:, mid_range_cols].sum(1)
        else:
            df[quantile_focus] = jdf.loc[:, high_flow_cols].sum(1)
    return df, jdf

def reformulate_dkl_model_result(model, metric, df, b):
    last_col = 2**b - 1
    col = f'{model}_{metric}'
        
    ddf = df[f'{col}_disagg'].str.split(', ', expand=True)
    ddf[0] = ddf[0].apply(lambda r: r.split('[')[1])
    ddf[last_col] = ddf[last_col].apply(lambda r: r.split(']')[0])
    
    for c in ddf.columns:
        ddf[c] = ddf[c].astype(float)
    ddf.columns = [f'{c:03d}' for c in ddf.columns]
    
    low_flow_cols = list(ddf.columns)[:2]
    high_flow_cols = list(ddf.columns)[-2:]
    mid_range_cols = [e for e in list(ddf.columns) if e not in low_flow_cols + high_flow_cols]
    for quantile_focus in [f'low_flow_{metric}_{model}', f'mid_range_{metric}_{model}', f'high_flow_{metric}_{model}']:
        if quantile_focus.startswith('low'):
            df[quantile_focus] = ddf.loc[:, low_flow_cols].sum(1)
        elif quantile_focus.startswith('mid'):
            df[quantile_focus] = ddf.loc[:, mid_range_cols].sum(1)
        else:
            df[quantile_focus] = ddf.loc[:, high_flow_cols].sum(1)
    return df, ddf


In [ ]:
class WeightHistory(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs=None):
        self.weight_history = []

    def on_epoch_end(self, epoch, logs=None):
        # Assuming model has a single layer or specifying the layer of interest
        weights = self.model.layers[0].get_weights()[0] # Adjust layer index as needed
        self.weight_history.append(weights)

In [ ]:
def prepare_model(dataframe, feature_columns, target_column):
    # Split the data into features and target
    X = dataframe[feature_columns].values
    y = dataframe[target_column].values
    print(f'{len(feature_columns)} feature columns')

    # Standardize the features
    # scaler = StandardScaler()
    scaler = QuantileTransformer(output_distribution='normal')
    X_scaled = scaler.fit_transform(X)

    # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

       # Define the model
    input_size = X_train.shape[1]
    model = keras.Sequential([
        layers.Dense(2, activation='relu', input_shape=(input_size,), kernel_initializer='he_uniform'), # input layer with (input size) features
        layers.Dropout(0.5), # dropout for regularization
        # layers.Dense(2, activation='relu'), # hidden layer
        layers.Dropout(0.5), # dropout for regularization
        layers.Dense(1)
    ])

    # Compile the model
    # model.compile(optimizer='adam', loss='mean_squared_error')
    model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mae'])

    return model, X_train, y_train, X_test, y_test

In [ ]:
HS_DATA_DIR = '/home/danbot2/code_5820/large_sample_hydrology/common_data/HYSETS_data/'
hs_properties_path = os.path.join(HS_DATA_DIR, 'HYSETS_watershed_properties.txt')
hs_df = pd.read_csv(hs_properties_path, sep=';')
hs_df.set_index('Official_ID', inplace=True)

In [ ]:
def add_coords(stn, col):
    return hs_df.loc[stn, col]    

In [ ]:
b = 6
# bin_model = 'equiprobable'
metric = 'tvd'
metric = 'dkl'
bin_model = 'uniform'
# metric = 'tvd'
target_column = f'{bin_model}_{metric}'
fname = f'compression_test_results_{b}bits_20240212.csv'
fname = f'DKL_test_results_{b}bits_20240212.csv'

In [ ]:
df = pd.read_csv(os.path.join('compression_test_results', fname))
print(f'{len(df)} samples in the results file')
# print(sorted(df.columns))
mcols = [c for c in df.columns if metric in c]
df[mcols].head()

In [ ]:
# log scale the drainage areas
da_cols = [c for c in df.columns if 'Drainage_Area' in c]
for c in da_cols:
    df[c] = np.log10(df[c].values)

In [ ]:
# df[[c for c in df.columns if 'jsd' in c]].head()
# df.columns
minc, maxc = min(df[target_column]), max(df[target_column])
print(f'{target_column} min: {minc:.2f} max: {maxc:.2f}')
neg_dkl = df[df[target_column] < 0].copy()
neg_dkl.head()

In [ ]:
# for c in ['Centroid_Lat_deg_N', 'Centroid_Lon_deg_E']:
#     df[f'proxy_{c}'] = df['proxy'].apply(lambda s: add_coords(s, c))
#     df[f'target_{c}'] = df['target'].apply(lambda s: add_coords(s, c))

# df.to_csv(os.path.join('compression_test_results', fname))

In [ ]:
# df, jdf = reformulate_model_result(model, df, b)
if metric != 'tvd':
    df, jdf = reformulate_dkl_model_result(bin_model, metric, df, b)

In [ ]:
feature_columns = [
    'Centroid_Lat_deg_N', 'Centroid_Lon_deg_E',
    'Drainage_Area_km2',
    'Elevation_m', 'Slope_deg',
    'Aspect_deg',
    'Gravelius', 'Perimeter',
    'Land_Use_Forest_frac', 'Land_Use_Grass_frac', 'Land_Use_Wetland_frac',
    'Land_Use_Snow_Ice_frac', 'Land_Use_Urban_frac', 'Land_Use_Shrubs_frac',
    'Land_Use_Crops_frac', 'Land_Use_Water_frac',
    'Permeability_logk_m2', 'Porosity_frac',
    'tmax', 'tmin',
    'prcp',
    'srad', 'swe', 'vp',
    'high_prcp_freq',
    'high_prcp_duration',
    'low_prcp_freq',
    'low_prcp_duration', 
]

feature_cols = [f'proxy_{c}' for c in feature_columns]
feature_cols += [f'target_{c}' for c in feature_columns]

# if target_column.startswith('low_flow'):
#     feature_columns = [e for e in feature_columns if not e.startswith('high_prcp')]
# elif target_column.startswith('high_flow'):
#     feature_columns = [e for e in feature_columns if not e.startswith('low_prcp')]


In [ ]:
input_df = df[feature_cols + ['centroid_distance'] + [target_column]].copy().dropna(how='any')

In [ ]:
model, X_train, y_train, X_test, y_test = prepare_model(input_df, feature_cols + ['centroid_distance'], target_column)

In [ ]:
from tensorflow.keras.callbacks import TensorBoard
import datetime

# Create a log directory with a unique name
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=3, verbose=1, mode='min', restore_best_weights=True)
weight_history_callback = WeightHistory()

In [ ]:
# Train the Model
epochs = 20 # Example number of epochs
cb_list = [tensorboard_callback, weight_history_callback, early_stopping_callback]
model.fit(X_train, y_train, epochs=epochs, callbacks=cb_list, 
          validation_split=0.2)

# Evaluate the Model
test_loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}')

In [ ]:
weight_matrix.shape

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

wh = weight_history_callback.weight_history
# Assuming `weight_history` is collected appropriately
epochs = range(len(wh))
features = range(len(wh[0]))

# Convert weight history to a 2D array for heatmap generation
weight_matrix = np.array([w.flatten() for w in wh])
plt.figure(figsize=(16, 5))
sns.heatmap(weight_matrix, yticklabels=epochs, cmap="viridis")
plt.xticks(rotation=45)
plt.xlabel('Feature')
plt.ylabel('Epoch')
plt.title(f'Heatmap of Weight Changes ({len(wh[0])} features)')
plt.show()

In [ ]:
# for i in range(len(wh)):
#     data = wh[i].flatten()
#     print(len(data))
#     indices_of_largest = np.argsort(np.abs(data))[-5:].flatten()
#     largest_weights = [data[n] for n in indices_of_largest]
#     feats = [feature_cols[n] for n in indices_of_largest]
#     notes = [f'{"_".join(f.split("_")[1:])}: {w:.2f}' for w, f in zip(largest_weights, feats)]
#     print(f'Epoch {i}: ', ', '.join(notes))

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir logs/fit

In [ ]:
# Assume 'model' is your trained model and X_test, y_test are your test data
predictions = model.predict(X_test)

In [ ]:
def create_vertical_histogram(residuals, bitrate, r, NBINS='auto'):
        
    # create the vertical histogram        
    vhist, vedges = np.histogram(residuals['residuals'], bins=NBINS, density=True)
    vzeros = np.zeros(len(vedges)-1)
    vmax = max(vhist)*1.1
    
    LINE_ARGS = dict(color="#3A5785", line_color=None)
    pv = figure(toolbar_location='above', width=200, height=350, x_range=(0, vmax),
                y_axis_location='right', y_range=r.y_range)
    pv.ygrid.grid_line_color = None
    pv.xaxis.major_label_orientation = np.pi/4
    pv.background_fill_color = "#fafafa"
    
    pv.quad(left=0, bottom=vedges[:-1], top=vedges[1:], right=vhist, color="white", line_color="#3A5785")
    vh1 = pv.quad(left=0, bottom=vedges[:-1], top=vedges[1:], right=vzeros, alpha=0.5, **LINE_ARGS)
    # vh2 = pv.quad(left=0, bottom=vedges[:-1], top=vedges[1:], right=vzeros, alpha=0.1, **LINE_ARGS)
    pv.line([0, max(vhist)], [0, 0], color='red', line_dash='dashed', line_width=3)
    # plot the 25 & 75 percentile lines
    pct1 = np.percentile(residuals['residuals'], 2.5)
    pct2 = np.percentile(residuals['residuals'], 98.5)
    pv.line([0, 0.1*max(vhist)], [pct1, pct1], color='orange', line_dash='dashed', line_width=3)
    pv.line([0, 0.1*max(vhist)], [pct2, pct2], color='orange', line_dash='dashed', line_width=3)
    pmed = np.percentile(residuals['residuals'], 50)
    pv.line([0, np.max(vhist)], [pmed, pmed], color='blue', line_dash='dashed', line_width=3)
    return pv

def create_horizontal_histogram(residuals, bitrate, r, NBINS='auto'):
        
    # create the vertical histogram        
    hist, edges = np.histogram(residuals['predicted'], bins=NBINS, density=True)
    zeros = np.zeros(len(edges)-1)
    hmax = max(hist)*1.1
    
    LINE_ARGS = dict(color="#3A5785", line_color=None)
    ph = figure(toolbar_location=None, width=400, height=200, y_range=(0, hmax),
                x_range=r.x_range)
    ph.ygrid.grid_line_color = None
    ph.xaxis.major_label_orientation = np.pi/4
    ph.background_fill_color = "#fafafa"
    
    ph.quad(left=edges[:-1], bottom=0, top=hist, right=edges[1:], color="white", line_color="#3A5785")
    h1 = ph.quad(left=edges[:-1], bottom=0, top=zeros, right=edges[1:], alpha=0.5, **LINE_ARGS)
    # vh2 = pv.quad(left=0, bottom=vedges[:-1], top=vedges[1:], right=vzeros, alpha=0.1, **LINE_ARGS)
    pmed = np.percentile(residuals['predicted'], 50)
    ph.line([pmed, pmed], [0, max(hist)], color='red', line_dash='dashed', line_width=3)
    # plot the 25 & 75 percentile lines
    # pct1 = np.percentile(residuals['predicted'], 2.5)
    # pct2 = np.percentile(residuals['predicted'], 98.5)
    # ph.line([pct1, pct1], [0, 0.1*max(hist)], color='orange', line_dash='dashed', line_width=3)
    # ph.line([0, 0.1*max(hist)], [pct2, pct2], color='orange', line_dash='dashed', line_width=3)
    # 
    # ph.line([0, np.max(hist)], [pmed, pmed], color='blue', line_dash='dashed', line_width=3)
    return ph

In [ ]:
def l1_best_fit(X, Y):
    """
    
    Compute the best fit line based on L1 norm (Least Absolute Deviations).
    
    :param X: Independent variable (e.g., actual values)
    :param Y: Dependent variable (e.g., predicted values)
    :return: Slope and intercept of the best fit line
    """
    # Objective function to minimize (sum of absolute differences)
    def objective_line(params):
        a, b = params
        return np.sum(np.abs(Y - (a * X + b)))

    # Initial guess for slope (a) and intercept (b)
    initial_guess = [0, np.median(Y)]

    # Minimize the objective function
    result = minimize(objective_line, initial_guess, method='Powell')

    # Perform linear regression
    slope, intercept, r_value, p_value, std_err = linregress(X, Y)

    return result.x, r_value**2, p_value


In [ ]:
def plot_confidence_band(x, y):
    if len(y) == 0:
        raise Exception('y is empty')
    rdf = pd.DataFrame({'x': x, 'y':y})
    results = []
    sorted_x = sorted(x)
    n_groups = 10
    for grp in np.array_split(sorted_x, n_groups):        
        grp_min, grp_max = min(grp), max(grp)
        data = rdf[(rdf['x'] >= grp_min) & (rdf['x'] < grp_max)].copy()
        y_vals = data['y'].values
        if len(y_vals) < 25:
            print(f'bin too small! ({grp_min}-{grp_max}) (N={len(y_vals)})')
        x_position = np.median(grp)
        median = np.median(y_vals)
        ci_lower = np.percentile(y_vals, 2.5)
        ci_upper = np.percentile(y_vals, 97.5)
        results.append((x_position, (grp_min + grp_max)/2, ci_lower, median, ci_upper))
    
    return results

In [ ]:
def bootstrap_variance_by_prediction(Y, X, n_bootstrap=1000, n_bins=20):
    """
    Estimates the variance of residuals as a function of predicted values
    using bootstrap resampling.

    Parameters:
    - p: array-like, predicted values.
    - s: array-like, observed (simulated) values.
    - n_bootstrap: int, number of bootstrap samples to generate.
    - n_bins: int, number of bins to divide the predicted values into.

    Returns:
    - A DataFrame with the bin centers, mean variance, and 95% confidence intervals for each bin.
    """
    # Calculate initial residuals
    residuals = X - Y

    equiprobable_bins = np.array_split(sorted(X), n_bins)
    bin_edges = [min(X)]
    b = 1
    bin_labels = []
    for grp in equiprobable_bins:
        bin_edges.append(max(grp))
        bin_labels.append(f'{bin_edges[b-1]:.2f}-{bin_edges[b]:.2f}')
        b += 1

    # Bin the predicted values
    bin_centers = np.add(bin_edges[:-1], bin_edges[1:]) / 2

    # DataFrame to store results
    results = pd.DataFrame({
        'bin_centre': bin_centers,
        'median': np.zeros(n_bins),
        'ci_lower': np.zeros(n_bins),
        'ci_upper': np.zeros(n_bins),
    })

    # Bootstrap resampling
    for i in range(n_bins):
        observed = []
        # Select observed values that fall into the current bin
        mask = (Y >= bin_edges[i]) & (Y < bin_edges[i+1])
        bin_vals = X[mask]

        for _ in range(n_bootstrap):
            # Resample observed values within the bin with replacement
            resample = np.random.choice(bin_vals, size=len(bin_vals), replace=True)
            observed.extend(resample)

        if observed:
            # Calculate 95% confidence interval from the bootstrap samples
            results.at[i, 'ci_lower'], results.at[i, 'ci_upper'] = np.percentile(observed, [2.5, 97.5])
            results.at[i, 'median'], results.at[i, 'bin_centre'] = np.percentile(observed, 50), bin_centers[i]
        else:
            # Handle bins with no data by setting NaN values
            results.at[i, 'ci_lower'] = np.nan
            results.at[i, 'ci_upper'] = np.nan
            
    results['Category'] = bin_labels
    return results

In [ ]:
# Define a hook function to rotate x-axis labels
def rotate_x_labels(plot, element):
    plot.state.xaxis.major_label_orientation = np.pi / 4  # 45 degrees in radians

def create_violin_plot(X, Y, n_bins=20):
    df = pd.DataFrame({'Predicted': X, 'Observed': Y})
    
    equiprobable_bins = np.array_split(sorted(X), n_bins)
    bin_edges = [min(X)]
    b = 1
    bin_labels = []
    for grp in equiprobable_bins:
        bin_edges.append(max(grp))
        bin_labels.append(f'{bin_edges[b-1]:.2f}-{bin_edges[b]:.2f}')
        b += 1
    
    df['Predicted Category'] = pd.cut(df['Predicted'], bins=bin_edges, labels=bin_labels)
    
    # Create a violin plot
    violin_plot = hv.Violin(df, ('Predicted Category', 'Predicted Bins'), 'Observed').sort()
    violin_plot = violin_plot.relabel(group='Quartiles').opts(opts.Violin(inner='quartiles', cut=1., bandwidth=1))
    violin_plot.opts(
        tools=['hover'], width=1000, height=400, 
        xlabel='Predicted Value Bins', ylabel='Observed Values',
        hooks=[rotate_x_labels])
    return violin_plot


In [ ]:
def plot_predictions_with_trendline(predicted, actual, bitrate, model):
    # Activate inline plotting in the notebook

    x1, x2 = min(predicted), max(predicted)
    y1, y2 = min(actual), max(actual)

    # get the L1 best fit parameters
    (m, b), r2, pval = l1_best_fit(predicted, actual)

    lx = np.linspace(x1, x2, 100)
    ly = [m*e + b for e in lx]

    # Create a new plot with a title and axis labels
    title = f"{bitrate} bit Predicted vs Observed {model} (N={len(actual)})"
    p = figure(title=title, width=400, height=350,
               y_axis_label=f'Observed {metric}', 
               x_axis_label=f'Predicted {metric}',
              # x_axis_type='log', y_axis_type='log'
              )
    
    # plot confidence interval
    plot_ci = False
    # plot_ci = True
    if plot_ci:
        # bin_medians, bin_midpoints, lb, median, ub = zip(*plot_confidence_band(predicted, actual))   
        var_df = bootstrap_variance_by_prediction(predicted, actual)
        vplot = create_violin_plot(predicted, actual)
        # bin_medians, bin_midpoints = var_df['bin_centre'], 
        # print(var_df.head())
        bin_medians = var_df['bin_centre'].values
        lb, ub, median = var_df['ci_lower'].values, var_df['ci_upper'].values, var_df['median'].values
        # print(asdfd)

    # Add scatter plot
    p.circle(predicted, actual, size=5, color="navy", alpha=0.25, legend_label='pts')

    vplot = None
    if plot_ci:
        # mx_adjusted = list(bin_midpoints)
        mx_adjusted = list(bin_medians)
        # mx_adjusted[0] -= 0.05
        p.varea(mx_adjusted, y1=lb, y2=ub, color='green', alpha=0.3, legend_label='95% CI')

    # plot the 1:1 line between actual vs. predicted
    p.line([0, max(predicted)], [0, max(predicted)], color='red', line_dash='dashed', line_width=3,
           legend_label='1:1')

    p.line(lx, ly, color='skyblue', line_dash='dashed', line_width=3,
           legend_label=f'R²={r2:.2f}')

    if plot_ci:
        p.circle(bin_medians, median, color='lawngreen', size=7, legend_label='median')
    p.legend.location = 'top_left'
    p.legend.background_fill_alpha = 0.7
    p.legend.click_policy = 'hide'

    title = f"{bin_model} Model Residuals (N={len(actual)}, {bitrate} bits)"
    r = figure(title=title, width=400, height=350,
               y_axis_label=f'Observed {bin_model}', 
               x_axis_label=f'Predicted {bin_model}',
              toolbar_location='above')

    residuals = pd.DataFrame()
    residuals['predicted'] = predicted
    residuals['observed'] = actual
    residuals['residuals'] = predicted - actual
    minr, maxr = residuals['observed'].min(), residuals['observed'].max()

    r.circle(residuals['predicted'], residuals['residuals'], 
             color='orange', size=2, alpha=0.4, )
    r.line([minr, maxr], [0, 0], color='red', line_width=3, line_dash='dashed')
    r.yaxis.axis_label = f'Residuals (pred - obs {metric})'
    r.xaxis.axis_label = f'Predicted {metric}'

    hp = create_vertical_histogram(residuals, bitrate, r)
    hh = create_horizontal_histogram(residuals, bitrate, r)
    
    return row(p, column(r, hh), hp), vplot

In [ ]:
plot, vplot = plot_predictions_with_trendline(predictions.flatten(), y_test, b, bin_model)
show(plot)

In [ ]:
plot, vplot = plot_predictions_with_trendline(predictions.flatten(), y_test, b, bin_model)
show(plot)

In [ ]:
plot, vplot = plot_predictions_with_trendline(predictions.flatten(), y_test, b, bin_model)
show(plot)

In [ ]:
plot, vplot = plot_predictions_with_trendline(predictions.flatten(), y_test, b, bin_model)
show(plot)

In [ ]:
def plot_feature_distributions(dataframe, num_cols=5):
    """
    Creates a grid plot of histograms for each feature in the dataframe using Bokeh.
    
    :param dataframe: A pandas DataFrame containing the normalized features.
    :param num_cols: Number of columns in the grid plot.
    """

    num_features = dataframe.shape[1]
    num_rows = np.ceil(num_features / num_cols).astype(int)

    plots = []
    for col in dataframe.columns:
        hist, edges = np.histogram(dataframe[col], bins=20, density=True)
        p = figure(title=f'Distribution of {col}', tools='', background_fill_color="#fafafa")
        p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], fill_color="navy", line_color="white", alpha=0.5)
        plots.append(p)
    return plots

In [ ]:
df.columns

In [ ]:
which_loc = 'proxy'
ccs = [f'{which_loc}_{c}' for c in feature_columns] + ['centroid_distance', target_column]
# ccs = [c.split('_frac')[0] for c in ccs]
print(ccs)

In [ ]:
# df[target_column] = np.sqrt(df[target_column])

ddf = df[ccs].copy().dropna()

ddf.head()
ddf['centroid_distance'].max()
# ddf['centroid_distance'] = np.sqrt(ddf['centroid_distance'])
scaled_df = (ddf[ccs] - ddf[ccs].min()) / (ddf[ccs].max() - ddf[ccs].min())
# ddf[['centroid_distance']].head()

In [ ]:
plots = plot_feature_distributions(scaled_df)
# Create a grid layout of plots
grid_layout = gridplot(plots, ncols=4, width=225, height=225)

# Show the grid layout
show(grid_layout)

In [ ]:
# plot the breakdown of JSD as a function of flow
b = 4
fname = f'compression_test_results_{b}bits_20240212.csv'
fname = f'DKL_test_results_{b}bits_20240212.csv'
df = pd.read_csv(os.path.join('compression_test_results',fname))
print(f'{len(df)} samples in the results file')

In [ ]:
model = 'equiprobable'
model = 'uniform'
df, jdf = reformulate_dkl_model_result(model, 'dkl', df, b)

In [ ]:
df_long = jdf.copy().reset_index().melt(id_vars='index')
df_long.columns = ['index', 'group', 'value']
df_long.head()

In [ ]:
x = sorted(list(set(jdf.columns)))
jdf.dropna(how='any', inplace=True)
medians = [np.percentile(jdf[c].values, 50) for c in jdf.columns]
lb = [np.percentile(jdf[c].values, 2.5) for c in jdf.columns]
ub = [np.percentile(jdf[c].values, 98.5) for c in jdf.columns]    

In [ ]:
source = ColumnDataSource(data=dict(base=x, upper=ub, lower=lb))
error = Whisker(base="base", upper="upper", lower="lower", source=source,
                level="annotation", line_width=0.5)
x_range = FactorRange(factors=x)

title=f"{model[0].upper() + model[1:]} KL Divergence by Quantization Level"

q = figure(width=600, height=350, title=title,
           output_backend='webgl', x_range=x_range)
q.circle(jitter("group", 0.3, range=x_range), "value", source=df_long,
         alpha=0.5, size=2, line_color="white",
         color='dodgerblue')
q.circle(x, medians, size=10)
q.add_layout(error)


show(q)

In [ ]:
source = ColumnDataSource(data=dict(base=x, upper=ub, lower=lb, median=medians))
error = Whisker(base="base", upper="upper", lower="lower", source=source,
                level="annotation", line_width=0.5)
x_range = FactorRange(factors=x)

title=f"{model[0].upper() + model[1:]} Jensen-Shannon Divergence by Quantization Level"

q = figure(width=600, height=350, title=title,
           output_backend='webgl', x_range=x_range)
q.circle('base', 'median', size=2, source=source)
# q.circle(jitter("group", 0.3, range=x_range), "value", source=df_long,
#          alpha=0.5, size=2, line_color="white",
#          color='dodgerblue')

q.add_layout(error)


show(q)
    